In [1]:
import boto3
from dotenv import load_dotenv
import os
import re
import paramiko
import requests

Don't forget to **not use https** in the url


In [3]:
load_dotenv()
aws_access_key_id= os.getenv("aws_access_key_id")
aws_secret_access_key = os.getenv("aws_secret_access_key")
region_name = os.getenv("region_name")

# Let's use Amazon S3
ec2 = boto3.client('ec2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

In [4]:
EC2_KWARGS = {
    "ImageId": "ami-07151644aeb34558a",
    "InstanceType": "t3.micro",
    "MinCount": 1,
    "MaxCount": 1,
    "KeyName": "Ec2-boto3",
    "SecurityGroupIds": ["sg-0b7b73b05e7577a3b"], #sg-02e30599ae3bdfd05
    "IamInstanceProfile":{
        'Arn': "arn:aws:iam::103976228435:instance-profile/EC2_S3_allowens"
    },
    "SubnetId": "subnet-0e19e3f6d404a4467",#"subnet-099718569ecc1d5ee",
    "Placement": {"AvailabilityZone": "eu-central-1a"},
    "BlockDeviceMappings": [
        {
            "DeviceName": "/dev/xvda",
            "Ebs": {
                "VolumeSize": 20,  # Größe des Root-Volumes in GB
                "DeleteOnTermination": True,
                "VolumeType": "gp3",
            },
        },
    ],
    "UserData": f"""#!/bin/bash
                sudo yum update -y
                sudo yum install docker -y
                sudo service docker start
                sudo usermod -a -G docker ec2-user
                sudo curl \
                    -L https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m) \
                    -o /usr/local/bin/docker-compose
                sleep 3    
                sudo chmod +x /usr/local/bin/docker-compose
             """,
    "TagSpecifications": [
        {
            "ResourceType": "instance",
            "Tags": [
                {"Key": "Name", "Value": "manager-instance"},
            ],
        },
    ],
    # "PrivateIpAddress":'10.0.0.10',
}

In [5]:
response = ec2.run_instances(**EC2_KWARGS)

# ID der gestarteten Instanz abrufen
instance_id = response['Instances'][0]['InstanceId']
print(f'Started instance {instance_id}')

Started instance i-08a3ee64c9c3f6b14


In [6]:
response = ec2.associate_address(
    InstanceId=instance_id,
    PublicIp='3.66.239.204'
)
response

{'AssociationId': 'eipassoc-0a9cf8427d3bd06de',
 'ResponseMetadata': {'RequestId': 'ea4f6184-eae0-449e-beb5-0f9400b1af48',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea4f6184-eae0-449e-beb5-0f9400b1af48',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '295',
   'date': 'Sat, 13 Jan 2024 09:58:51 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

look up the public ip of the ec2 instance in the aws console

In [7]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('3.66.239.204', username='ec2-user', key_filename='Ec2-boto3.pem')

In [19]:
# Kopieren Sie die Docker Compose-Datei auf die Instanz
sftp = ssh.open_sftp()
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\.env', '/home/ec2-user/.env')
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\nginx/nginx.conf', '/home/ec2-user/nginx.conf')
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\docker-compose.yml', '/home/ec2-user/docker-compose.yml')
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\scripts\init-swarm.sh', '/home/ec2-user/init-swarm.sh')
sftp.put(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\src\scripts\run-container.sh', '/home/ec2-user/run-container.sh')
sftp.close()

execute the following commands on the ec2 instance
```bash
sudo bash init-swarm.sh
```

In [8]:
stdin, stdout, stderr = ssh.exec_command('sudo docker-compose up -d')

In [10]:
print(stdout)

<paramiko.ChannelFile from <paramiko.Channel 1 (closed) -> <paramiko.Transport at 0x25e11cf0 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>>>


### Start an RDS instance for the database

In [9]:
rds_client = boto3.client("rds", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

In [12]:
response = rds_client.create_db_instance(
    DBName="SolarParkDetection",
    DBInstanceIdentifier=os.getenv("POSTGRES_DB"),
    AllocatedStorage=20,
    DBInstanceClass="db.t4g.micro",
    Engine="postgres",
    MasterUsername=os.getenv("POSTGRES_USER"),
    MasterUserPassword=os.getenv("POSTGRES_PASSWORD"),
    VpcSecurityGroupIds=[
        "sg-0b7b73b05e7577a3b",
    ],
    AvailabilityZone="eu-central-1a",
    Tags=[
        {
            "Key": "Name",
            "Value": "postgres-instance",
        },
    ],
    PubliclyAccessible=True,
)
response


DBInstanceAlreadyExistsFault: An error occurred (DBInstanceAlreadyExists) when calling the CreateDBInstance operation: DB instance already exists

In [14]:
# Get the RDS instance endpoint
instances = rds_client.describe_db_instances(DBInstanceIdentifier=os.getenv("POSTGRES_DB"))
rds_host = instances.get('DBInstances')[0].get('Endpoint').get('Address')
rds_host

'solarparkdetection.cwoh0q12sshz.eu-central-1.rds.amazonaws.com'

#### Additionally, we need to install the PostGIS extension on the database:

In [15]:

import psycopg2

# Replace these with your own values
host = rds_host
port = "5432"
dbname = "SolarParkDetection"
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")

# Connect to the database
conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

# Create a cursor object
cur = conn.cursor()

# Execute the SQL command to create the PostGIS extension
cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
cur.execute("CREATE EXTENSION IF NOT EXISTS postgis_topology;")
cur.execute("CREATE EXTENSION IF NOT EXISTS fuzzystrmatch;")
cur.execute("CREATE EXTENSION IF NOT EXISTS postgis_tiger_geocoder;")

# Commit the transaction
conn.commit()

In [16]:
# check if extension is installed
cur.execute("SELECT * FROM pg_extension;")

In [17]:
# Close the cursor and the connection
cur.close()
conn.close()